- This one is based on the great starting [notebook](https://www.kaggle.com/code/nbroad/transformer-ner-baseline-lb-0-576) by @NICHOLAS BROAD.
- please note: I didn't use the deberta-base model I choose here to finetune, this is just a demo.
- In the postprocessing part of inference, I changed **tokens** to **token_map** in two lines. This will bring a huge improvement. There is the [inference notebook](https://www.kaggle.com/code/takanashihumbert/piidd-deberta-model-starter-inference)

#メモ

In [67]:
"""
メモ
tokenizerで，strideとmax_lenを指定することで，strideの長さのトークン分被るようにして，それぞれmax_lenの長さでシーケンスを分割できる．
これを，documentとどこのtokenに対応するかを覚えておく．これで各1024に対し，予測していく．

#構成
最初にtokenizeでmax_lenの長さにtextをばらす．
"""

'\nメモ\ntokenizerで，strideとmax_lenを指定することで，strideの長さのトークン分被るようにして，それぞれmax_lenの長さでシーケンスを分割できる．\nこれを，documentとどこのtokenに対応するかを覚えておく．これで各1024に対し，予測していく．\n\n#構成\n最初にtokenizeでmax_lenの長さにtextをばらす．\n'

In [68]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [69]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import runtime

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Config

In [70]:
class CFG:
  exp = '1st_exp001'
  n_folds = 4
  trn_folds = [-1]
  max_len = 512
  stride = 128
  model_name = 'microsoft/deberta-v3-large'
  seed=42
  target = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM',
    'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM',
    'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL'
  ]
  DEBUG=False



In [71]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = f'/content/drive/MyDrive/Kaggle/competition/PIIDD/outputs/{CFG.exp}/'

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

input_path = '/content/drive/MyDrive/Kaggle/competition/PIIDD/input/'

# Import

In [72]:
!pip install seqeval evaluate -q
!pip install sentencepiece

!pip install transformers==4.31.0
os.system('pip install -q tokenizers')
!pip install tokenizers==0.13.3

!pip install accelerate -U

import random
import gc
import json
import argparse
from itertools import chain
from functools import partial

import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
import evaluate
from datasets import Dataset, features, DatasetDict
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold,StratifiedKFold

from scipy.special import softmax
from torch.nn import CrossEntropyLoss
from datasets import load_dataset, concatenate_datasets

# seed

In [73]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=CFG.seed)

In [74]:
reference_data = json.load(open(input_path+"/train.json", "r"))

for i in range(len(reference_data)):
  reference_data[i]['document']=i+1


df = pd.DataFrame(reference_data)[['document', 'tokens', 'labels']].copy()
df = df.explode(['tokens', 'labels']).reset_index(drop=True).rename(columns={'tokens': 'token', 'labels': 'label'})
df['token_str']=df['token']
df['token'] = df.groupby('document').cumcount()

label_list = df['label'].unique().tolist()

reference_df = df[df['label'] != 'O'].copy()
reference_df_with_O = df.copy()



reference_df.to_csv(OUTPUT_DIR+f'reference.csv',index=False)

In [75]:
search_dict = {}
for row in reference_df_with_O.itertuples():
  search_dict[(row.document,row.token)]=row.token_str

In [76]:
# make random predictions on a subset of tokens
rng = random.Random(42)
pred_df = reference_df.sample(frac=0.8).sort_values(by='document').reset_index(drop=True)
pred_df['label'] = pred_df['label'].apply(lambda x: rng.choice(label_list) if rng.random() >= 0.5 else x)
pred_df['row_id'] = list(range(len(pred_df)))
pred_df = pred_df[['row_id', 'document', 'token', 'label']].copy()

pred_df

,row_id,document,token,label
0,0,1,742,O
1,1,1,9,B-URL_PERSONAL
2,2,1,482,B-NAME_STUDENT
3,3,1,483,B-STREET_ADDRESS
4,4,2,465,B-PHONE_NUM
...,...,...,...,...
2186,2186,3770,964,B-ID_NUM
2187,2187,3770,365,I-NAME_STUDENT
2188,2188,5240,55,I-ID_NUM
2189,2189,5240,54,B-EMAIL


In [77]:
from collections import defaultdict
from typing import Dict


class PRFScore:
    """A precision / recall / F score."""

    def __init__(
        self,
        *,
        tp: int = 0,
        fp: int = 0,
        fn: int = 0,
    ) -> None:
        self.tp = tp
        self.fp = fp
        self.fn = fn

    def __len__(self) -> int:
        return self.tp + self.fp + self.fn

    def __iadd__(self, other):  # in-place add
        self.tp += other.tp
        self.fp += other.fp
        self.fn += other.fn
        return self

    def __add__(self, other):
        return PRFScore(
            tp=self.tp + other.tp, fp=self.fp + other.fp, fn=self.fn + other.fn
        )

    def score_set(self, cand: set, gold: set) -> None:
        self.tp += len(cand.intersection(gold))
        self.fp += len(cand - gold)
        self.fn += len(gold - cand)

    @property
    def precision(self) -> float:
        return self.tp / (self.tp + self.fp + 1e-100)

    @property
    def recall(self) -> float:
        return self.tp / (self.tp + self.fn + 1e-100)

    @property
    def f1(self) -> float:
        p = self.precision
        r = self.recall
        return 2 * ((p * r) / (p + r + 1e-100))

    @property
    def f5(self) -> float:
        beta = 5
        p = self.precision
        r = self.recall

        fbeta = (1+(beta**2))*p*r / ((beta**2)*p + r + 1e-100)
        return fbeta

    def to_dict(self) -> Dict[str, float]:
        return {"p": self.precision, "r": self.recall, "f5": self.f5}


def eval_metrics(pred_df, gt_df):
    """
    Compute the LB metric (lb) and other auxiliary metrics
    """

    references = {(row.document, row.token, row.label) for row in gt_df.itertuples()}
    predictions = {(row.document, row.token, row.label) for row in pred_df.itertuples()}

    score_per_type = defaultdict(PRFScore)
    references = set(references)

    for ex in predictions:
        pred_type = ex[-1] # (document, token, label)
        # if pred_type != 'O':
        #     pred_type = pred_type[2:] # avoid B- and I- prefix

        if pred_type not in score_per_type:
            score_per_type[pred_type] = PRFScore()

        if ex in references:
            score_per_type[pred_type].tp += 1
            references.remove(ex)
        else:
            score_per_type[pred_type].fp += 1

    for doc, tok, ref_type in references:
        # if ref_type != 'O':
        #     ref_type = ref_type[2:] # avoid B- and I- prefix

        if ref_type not in score_per_type:
            score_per_type[ref_type] = PRFScore()
        score_per_type[ref_type].fn += 1

    totals = PRFScore()

    for prf in score_per_type.values():
        totals += prf

    return {
        "ents_p": totals.precision,
        "ents_r": totals.recall,
        "ents_f5": totals.f5,
    }

# Cross Validation


In [326]:
def creates_folds(data,num_splits):
  '''
  Parameters:
  - data: (json) This is only piidd dataset.
  - num_splits: (int)

  Returns:
  - data: (json)
  '''
  labels = [item['is_label'] for item in data]
  kf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=CFG.seed)
  cnt=[0]*(num_splits)
  for i,(_t, _v) in  enumerate(kf.split(data,labels)):
    for idx in _v:
      data[idx]['fold']=i
  return data


data = json.load(open(input_path+"train.json"))

for i in range(len(data)):
  flag=True
  for label in data[i]['labels']:
      if label in CFG.target:
          data[i]['is_label']=1
          flag=False
          break
  if flag:
    data[i]['is_label']=0

data=creates_folds(data,CFG.n_folds)


external_data = json.load(open(input_path+"mixtral-8x7b-v1.json"))
print(len(external_data))
print(external_data[0].keys())

map_dict={}

for i in range(len(external_data)):
  external_data[i]['fold']=-1
data = data+external_data

for i in range(len(data)):
  map_dict[data[i]['document']]=i
  data[i]['document']=i+1


# del external_data1,external_data
gc.collect()

print(len(data))
print(data[0].keys())


2355
dict_keys(['document', 'full_text', 'tokens', 'labels', 'trailing_whitespace'])
9162
dict_keys(['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels', 'is_label', 'fold'])


In [327]:
df = pd.DataFrame(data)[['document', 'tokens', 'labels','fold']].copy()
df = df.explode(['tokens', 'labels']).reset_index(drop=True).rename(columns={'tokens': 'token', 'labels': 'label'})

df['token'] = df.groupby('document').cumcount()

label_list = df['label'].unique().tolist()

reference_df = df[(df['label'] != 'O')&(df['fold']!=-1)].copy()

In [328]:
# pred_df_040=pd.read_csv('/content/drive/MyDrive/Kaggle/competition/PIIDD/outputs/1st_exp001/preds_0.97_-1.csv')[['document','token','label']]
# pred_df_040['label'] = pred_df_040['label'].str[2:]

In [329]:
# pred_df_041=pd.read_csv('/content/drive/MyDrive/Kaggle/competition/PIIDD/outputs/exp046/preds_0.98_-1.csv')[['document','token','label']]
# pred_df_041['label'] = pred_df_041['label'].str[2:]

In [330]:
pred_df_042=pd.read_csv('/content/drive/MyDrive/Kaggle/competition/PIIDD/outputs/exp042/preds_0.99_-1.csv')[['document','token','label']]
pred_df_042['label'] = pred_df_042['label'].str[2:]

In [331]:
# pred_df_043=pd.read_csv('/content/drive/MyDrive/Kaggle/competition/PIIDD/outputs/exp043_freeze/preds_0.98_-1.csv')[['document','token','label']]
# pred_df_043['label'] = pred_df_043['label'].str[2:]

In [332]:
pred_df_044=pd.read_csv('/content/drive/MyDrive/Kaggle/competition/PIIDD/outputs/exp044/preds_0.97_-1.csv')[['document','token','label']]
pred_df_044['label'] = pred_df_044['label'].str[2:]

In [333]:

pred_df_055=pd.read_csv('/content/drive/MyDrive/Kaggle/competition/PIIDD/outputs/exp055/preds_0.97_-1.csv')[['document','token','label']]
pred_df_055['label'] = pred_df_055['label'].str[2:]

In [334]:
pred_df_058=pd.read_csv('/content/drive/MyDrive/Kaggle/competition/PIIDD/outputs/exp058/preds_0.97_-1.csv')[['document','token','label']]
pred_df_058['label'] = pred_df_058['label'].str[2:]

In [335]:
# # それぞれのデータフレームから 'document' と 'token' の列を取得し、重複を除外して結合する
unique_tokens = pd.concat([pred_df_055[['document', 'token']], pred_df_042[['document', 'token']], pred_df_058[['document', 'token']],pred_df_044[['document', 'token']]]).drop_duplicates()



def fill_O_df(pred_df, unique_tokens):
  # 'document' と 'token' の組み合わせをキーとして重複を除外し、それ以外の列を無視する
  merged_df = pd.merge(unique_tokens, pred_df, on=['document', 'token'], how='left', indicator=True)
  # 'pred_df_041' にしか存在しない行を抽出
  rows_only = merged_df[merged_df['_merge'] == 'left_only']

  # 'label' 列に 'O' を追加して抽出した行を新しいデータフレームに追加
  new_rows = rows_only.assign(label='O')[['document', 'token', 'label']]

  # 元のデータフレームに新しい行を追加
  pred_df = pd.concat([pred_df, new_rows], ignore_index=True)
  return pred_df

pred_df_055=fill_O_df(pred_df_055,unique_tokens)
pred_df_042=fill_O_df(pred_df_042,unique_tokens)
pred_df_058=fill_O_df(pred_df_058,unique_tokens)
pred_df_044=fill_O_df(pred_df_044,unique_tokens)

In [336]:
pred_df=pd.concat([pred_df_055,pred_df_044,pred_df_042,pred_df_055],axis=0)

In [337]:
# pred_df=pred_df_058

In [338]:
grouped_df = pred_df.groupby(['token', 'document', 'label']).size().reset_index(name='count')

# カウントが最大のグループを抽出する
pred_df = grouped_df.loc[grouped_df.groupby(['token', 'document'])['count'].idxmax()]


pred_df=pred_df.drop_duplicates()

# 'document' および 'token' 列でデータフレームをソート
pred_df = pred_df.sort_values(by=['document', 'token'])

In [339]:
pred_df=pred_df[pred_df['label']!='O']

In [340]:
pred_df

,token,document,label,count
957,6,1,NAME_STUDENT,2
1333,9,1,NAME_STUDENT,4
1432,10,1,NAME_STUDENT,4
4150,479,1,NAME_STUDENT,2
4176,482,1,NAME_STUDENT,4
...,...,...,...,...
5614,884,6580,NAME_STUDENT,4
4841,628,6630,NAME_STUDENT,2
4965,653,6630,NAME_STUDENT,2
2635,185,6675,NAME_STUDENT,4


In [341]:
# for row in pred_df.itertuples():
#   if row.label=='NAME_STUDENT':
#     tmp_df=reference_df_with_O[(reference_df_with_O['token_str']==row.token_str)&(reference_df_with_O['document']==row.document)]
#     pred_df=pd.concat([pred_df,tmp_df],axis=0)


In [342]:
reference_df_with_O

,document,token,label,token_str
0,1,0,O,Design
1,1,1,O,Thinking
2,1,2,O,for
3,1,3,O,innovation
4,1,4,O,reflexion
...,...,...,...,...
4992528,6807,815,O,process
4992529,6807,816,O,explained
4992530,6807,817,O,above
4992531,6807,818,O,.


In [343]:
pred_df=pd.merge(pred_df,reference_df_with_O[['document','token','token_str']],on=['document','token'],how='left')

In [344]:
# # labelがNAME_STUDENTの行をフィルタリング
# student_rows = pred_df[pred_df['label'] == 'NAME_STUDENT']
# student_rows.drop('token',axis=1,inplace=True)



In [345]:
# # reference_df_with_Oから該当行を抽出
# # 同じdocument列とtoken_str列を持つreference_df_with_Oから該当行を抽出
# matched_rows = student_rows.merge(reference_df_with_O, on=['document', 'token_str'], how='inner')

In [346]:
# add_df=matched_rows[['document','token','token_str']].drop_duplicates()
# add_df['label']='NAME_STUDENT'

In [347]:

# add_df

In [348]:
# pred_df=pd.concat([pred_df,add_df],axis=0)
# pred_df.drop('count',axis=1,inplace=True)

In [349]:
# pred_df=pred_df.drop_duplicates()

In [350]:
pred_df

,token,document,label,count,token_str
0,6,1,NAME_STUDENT,2,Avril
1,9,1,NAME_STUDENT,4,Nathalie
2,10,1,NAME_STUDENT,4,Sylla
3,479,1,NAME_STUDENT,2,Avril
4,482,1,NAME_STUDENT,4,Nathalie
...,...,...,...,...,...
3888,884,6580,NAME_STUDENT,4,Madelon
3889,628,6630,NAME_STUDENT,2,Shultz
3890,653,6630,NAME_STUDENT,2,Shultz
3891,185,6675,NAME_STUDENT,4,Rita


In [351]:
ban_set=set()
now_token=-1
now_doc=-1

for row in pred_df.itertuples():
  if row.token>0 and row.label=='NAME_STUDENT':
    if now_token==row.token-1 and now_doc==row.document:
      ban_set.add((row.document,row.token_str))
      now_token=row.token
      now_doc=row.document

    if (row.document,row.token+1) in search_dict:
      if search_dict[(row.document,row.token-1)] in ['The','the', 'a','an','at','my','My'] and search_dict[(row.document,row.token+1)]!='\'s':
        ban_set.add((row.document,row.token_str))
        now_token=row.token
        now_doc=row.document
    else:
      if search_dict[(row.document,row.token-1)] in ['The','the', 'a','an','at','my','My']:
        ban_set.add((row.document,row.token_str))
        now_token=row.token
        now_doc=row.document
pred_df = pred_df[~pred_df[['document', 'token_str']].apply(tuple, axis=1).isin(ban_set)]


In [352]:
pred_df = pred_df.sort_values(by=['document', 'token']).drop(columns=['count'])
pred_df

,token,document,label,token_str
0,6,1,NAME_STUDENT,Avril
1,9,1,NAME_STUDENT,Nathalie
2,10,1,NAME_STUDENT,Sylla
3,479,1,NAME_STUDENT,Avril
4,482,1,NAME_STUDENT,Nathalie
...,...,...,...,...
3888,884,6580,NAME_STUDENT,Madelon
3889,628,6630,NAME_STUDENT,Shultz
3890,653,6630,NAME_STUDENT,Shultz
3891,185,6675,NAME_STUDENT,Rita


In [353]:
pred_cnt_df=pred_df.groupby(['document','token_str']).size().reset_index(name='count')

In [354]:
ref_cnt_df=reference_df_with_O.groupby(['document','token_str']).size().reset_index(name='count_ref')

In [355]:
pred_df=pd.merge(pred_df,pred_cnt_df,on=['document','token_str'],how='left')
pred_df=pd.merge(pred_df,ref_cnt_df,on=['document','token_str'],how='left')

In [356]:
pred_df

,token,document,label,token_str,count,count_ref
0,6,1,NAME_STUDENT,Avril,3,3
1,9,1,NAME_STUDENT,Nathalie,3,3
2,10,1,NAME_STUDENT,Sylla,3,3
3,479,1,NAME_STUDENT,Avril,3,3
4,482,1,NAME_STUDENT,Nathalie,3,3
...,...,...,...,...,...,...
3813,884,6580,NAME_STUDENT,Madelon,5,5
3814,628,6630,NAME_STUDENT,Shultz,2,3
3815,653,6630,NAME_STUDENT,Shultz,2,3
3816,185,6675,NAME_STUDENT,Rita,1,1


In [357]:
reference_df=reference_df.rename(columns={'label':'ref_label'}).drop(columns=['fold'])

In [358]:
pred_df=pd.merge(pred_df,reference_df,on=['document','token'],how='left')

In [359]:
# pred_df['cnt_ratio']=pred_df['count']/pred_df['count_ref']

In [360]:
# pred_df['ref_label']=pred_df.fillna('n-False')['ref_label'].str[2:]

In [361]:
# reference_df_with_O['doc_token_str']=reference_df_with_O['document'].astype(str)+'_'+reference_df_with_O['token_str']

In [362]:
# pred_df['doc_token_str']=pred_df['document'].astype(str)+'_'+pred_df['token_str']

In [363]:
# a=pred_df[(pred_df['label']=='NAME_STUDENT')&(pred_df['cnt_ratio']>0.8)&(pred_df['cnt_ratio']<1.0)][['document','token_str']].drop_duplicates()
# a['doc_token_str']=a['document'].astype(str)+'_'+a['token_str']
# pred_df=pred_df[~pred_df['doc_token_str'].isin(a['doc_token_str'])]
# add_df=reference_df_with_O[reference_df_with_O['doc_token_str'].isin(a['doc_token_str'])][['document','token','label']]
# add_df['label']='NAME_STUDENT'
# add_df

In [364]:
# a=pred_df[(pred_df['label']=='NAME_STUDENT')&(pred_df['cnt_ratio']<0.2)][['document','token_str']].drop_duplicates()
# a['doc_token_str']=a['document'].astype(str)+'_'+a['token_str']
# # reference_df_with_O[reference_df_with_O['doc_token_str'].isin(a['doc_token_str'])]

In [365]:
# pred_df=pred_df[~pred_df['doc_token_str'].isin(a['doc_token_str'])]

In [366]:
# pred_df=pred_df[['document','token','label']]
# pred_df=pd.concat([pred_df,add_df],axis=0)

In [367]:
pred_df = pred_df.sort_values(by=['document', 'token'])

In [368]:
pred_df

,token,document,label,token_str,count,count_ref,ref_label
0,6,1,NAME_STUDENT,Avril,3,3,NaN
1,9,1,NAME_STUDENT,Nathalie,3,3,B-NAME_STUDENT
2,10,1,NAME_STUDENT,Sylla,3,3,I-NAME_STUDENT
3,479,1,NAME_STUDENT,Avril,3,3,NaN
4,482,1,NAME_STUDENT,Nathalie,3,3,B-NAME_STUDENT
...,...,...,...,...,...,...,...
3813,884,6580,NAME_STUDENT,Madelon,5,5,NaN
3814,628,6630,NAME_STUDENT,Shultz,2,3,NaN
3815,653,6630,NAME_STUDENT,Shultz,2,3,NaN
3816,185,6675,NAME_STUDENT,Rita,1,1,NaN


In [369]:
# 初期化
prev_doc = None
prev_token = None
prev_label = None

# 'label' 列の処理
for index, row in pred_df.iterrows():
    if row['document'] == prev_doc and row['token'] == prev_token + 1 and row['label'] == prev_label:
        # 同じ document かつ連続している token の場合、I- を付ける
        pred_df.loc[index, 'label'] = 'I-' + row['label']
    else:
        # 初めての document または非連続な token の場合、B- を付ける
        pred_df.loc[index, 'label'] = 'B-' + row['label']


    # 前の行の情報を更新
    prev_doc = row['document']
    prev_token = row['token']
    prev_label = row['label']


In [370]:
reference_df.rename(columns={'ref_label':'label'},inplace=True)

In [371]:
eval_metrics(pred_df.copy(),reference_df.copy())

# {'ents_p': 0.7113672079622839,
#  'ents_r': 0.9916027747353049,
#  'ents_f5': 0.976802733321345}
# pred_df=pd.concat([pred_df_055,pred_df_044,pred_df_042,pred_df_055],axis=0)



{'ents_p': 0.7113672079622839,
 'ents_r': 0.9916027747353049,
 'ents_f5': 0.976802733321345}

In [372]:
eval_metrics(pred_df.copy(),reference_df.copy())

{'ents_p': 0.7113672079622839,
 'ents_r': 0.9916027747353049,
 'ents_f5': 0.976802733321345}

In [373]:
# eval_metrics(sub_df.copy(),reference_df.copy())

# id2label

In [374]:
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

print(id2label)

{0: 'B-EMAIL', 1: 'B-ID_NUM', 2: 'B-NAME_STUDENT', 3: 'B-PHONE_NUM', 4: 'B-STREET_ADDRESS', 5: 'B-URL_PERSONAL', 6: 'B-USERNAME', 7: 'I-ID_NUM', 8: 'I-NAME_STUDENT', 9: 'I-PHONE_NUM', 10: 'I-STREET_ADDRESS', 11: 'I-URL_PERSONAL', 12: 'O'}


In [375]:
id2label

{0: 'B-EMAIL',
 1: 'B-ID_NUM',
 2: 'B-NAME_STUDENT',
 3: 'B-PHONE_NUM',
 4: 'B-STREET_ADDRESS',
 5: 'B-URL_PERSONAL',
 6: 'B-USERNAME',
 7: 'I-ID_NUM',
 8: 'I-NAME_STUDENT',
 9: 'I-PHONE_NUM',
 10: 'I-STREET_ADDRESS',
 11: 'I-URL_PERSONAL',
 12: 'O'}

In [376]:
def is_nested_list(lst):
    return any(isinstance(item, list) for item in lst)

In [377]:
def tokenize(example, tokenizer, label2id):
    text = []
    token_map = []
    # these are at the character level
    labels = []
    targets = []
    idx=0

    for t, l, ws in zip(example["tokens"], example["provided_labels"], example["trailing_whitespace"]):

        text.append(t)
        #文字レベルでlabelを振っている．
        labels.extend([l]*len(t))
        token_map.extend([idx]*len(t))

        if l in CFG.target:
            targets.append(1)
        else:
            targets.append(0)
        # if there is trailing whitespace
        if ws:
            text.append(" ")
            labels.append("O")
            token_map.append(-1)
        idx += 1

    target_num = sum(targets)
    labels = np.array(labels)

    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=CFG.max_len, stride=CFG.stride,return_overflowing_tokens=True)
    text = "".join(text)

    token_samples=[]

    if not is_nested_list(tokenized.offset_mapping):
      token_labels = []
      for start_idx, end_idx in tokenized.offset_mapping:

          # CLS token
          if start_idx == 0 and end_idx == 0:
              token_labels.append(label2id["O"])
              continue

          # case when token starts with whitespace
          if text[start_idx].isspace() and text[start_idx]!='\n':
              start_idx += 1

          token_labels.append(label2id[labels[start_idx]])

      length = len(tokenized.input_ids)

      return [
          tokenized.input_ids,
          tokenized.token_type_ids,
          tokenized.attention_mask,
          tokenized.offset_mapping,
          tokenized.overflow_to_sample_mapping,
          token_labels,
          length,
          target_num,
          1 if target_num>0 else 0,
          example['document'],
          example['fold'],
          token_map,
          example["tokens"]
      ]

    else:
      for i in range(len(tokenized.offset_mapping)):
        token_labels = []

        for start_idx, end_idx in tokenized.offset_mapping[i]:

            # CLS token
            if start_idx == 0 and end_idx == 0:
                token_labels.append(label2id["O"])
                continue

            # case when token starts with whitespace
            if text[start_idx].isspace() and text[start_idx]!='\n':
                start_idx += 1

            token_labels.append(label2id[labels[start_idx]])

        length = len(tokenized.input_ids[i])

        token_sample= [
          tokenized.input_ids[i],
          tokenized.token_type_ids[i],
          tokenized.attention_mask[i],
          tokenized.offset_mapping[i],
          tokenized.overflow_to_sample_mapping[i],
          token_labels,
          length,
          target_num,
          1 if target_num>0 else 0,
          example['document'],
          example['fold'],
          token_map,
          example["tokens"]
      ]
        token_samples.append(token_sample)

      return token_samples

## competition metrics

In [378]:
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

def parse_predictions(predictions, id2label, ds,fold,threshold=0.9):

    pred_softmax = np.exp(predictions) / np.sum(np.exp(predictions), axis = 2).reshape(predictions.shape[0],predictions.shape[1],1)
    preds = predictions.argmax(-1)
    preds_without_O = pred_softmax[:,:,:12].argmax(-1)
    O_preds = pred_softmax[:,:,12]
    preds_final = np.where(O_preds < threshold, preds_without_O , preds)

    triplets = []
    row, document, token, label, token_str = [], [], [], [], []
    for i, (p, token_map, offsets, tokens, doc) in enumerate(zip(preds_final, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"])):

        for token_pred, (start_idx, end_idx) in zip(p, offsets):
            label_pred = id2label[token_pred]

            if start_idx + end_idx == 0: continue

            if token_map[start_idx] == -1:
                start_idx += 1

            # ignore "\n\n"
            while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace() and tokens[token_map[start_idx]]!='\n':
                start_idx += 1

            if start_idx >= len(token_map): break

            token_id = token_map[start_idx]

            # ignore "O" predictions and whitespace preds
            if label_pred != "O" and token_id != -1:
                triplet = (label_pred, token_id, tokens[token_id])
                #ここを改善

                row.append(i)
                document.append(doc)
                token.append(token_id)
                label.append(label_pred)
                token_str.append(tokens[token_id])
                triplets.append(triplet)

    df = pd.DataFrame({
        "eval_row": row,
        "document": document,
        "token": token,
        "label": label,
        "token_str": token_str
    })



    # tokenとdocumentでグループ化し、各グループ内のlabelの個数を数える
    grouped_df = df.groupby(['token', 'document', 'label']).size().reset_index(name='count')

    # カウントが最大のグループを抽出する
    df = grouped_df.loc[grouped_df.groupby(['token', 'document'])['count'].idxmax()]

    df = df.drop_duplicates().reset_index(drop=True)

    df["row_id"] = list(range(len(df)))
    df.to_csv(OUTPUT_DIR+f'preds_{threshold}_{fold}.csv',index=False)
    return df


def compute_metrics(p, id2label, valid_ds, valid_df,fold,threshold=0.97):
    """
    Compute the LB metric (lb) and other auxiliary metrics
    """
    predictions, labels = p

    pred_df = parse_predictions(predictions, id2label, valid_ds,fold,threshold=threshold)

    references = {(row.document, row.token, row.label) for row in valid_df.itertuples()}
    predictions = {(row.document, row.token, row.label) for row in pred_df.itertuples()}

    score_per_type = defaultdict(PRFScore)
    references = set(references)

    for ex in predictions:
        pred_type = ex[-1] # (document, token, label)
        # if pred_type != 'O':
        #     pred_type = pred_type[2:] # avoid B- and I- prefix

        if pred_type not in score_per_type:
            score_per_type[pred_type] = PRFScore()

        if ex in references:
            score_per_type[pred_type].tp += 1
            references.remove(ex)
        else:
            score_per_type[pred_type].fp += 1

    for doc, tok, ref_type in references:
        # if ref_type != 'O':
        #     ref_type = ref_type[2:] # avoid B- and I- prefix

        if ref_type not in score_per_type:
            score_per_type[ref_type] = PRFScore()
        score_per_type[ref_type].fn += 1

    totals = PRFScore()

    for prf in score_per_type.values():
        totals += prf


    results = {
        "ents_p": totals.precision,
        "ents_r": totals.recall,
        "ents_f5": totals.f5,
    }
    return results

    # results = {
    #     "ents_p": totals.precision,
    #     "ents_r": totals.recall,
    #     "ents_f5": totals.f5,
    #     "ents_per_type": {k: v.to_dict() for k, v in score_per_type.items() if k!= 'O'},
    # }

    # # Unpack nested dictionaries
    # final_results = {}
    # for key, value in results.items():
    #     if isinstance(value, dict):
    #         for n, v in value.items():
    #             if isinstance(v, dict):
    #                 for n2, v2 in v.items():
    #                     final_results[f"{key}_{n}_{n2}"] = v2
    #             else:
    #                 final_results[f"{key}_{n}"] = v
    #     else:
    #         final_results[key] = value

    # return final_results

In [379]:
def freeze_layer(model,FREEZE_LAYERS = 6,FREEZE_EMBEDDINGS = False):
  if FREEZE_EMBEDDINGS:
      print('Freezing embeddings.')
      for param in model.deberta.embeddings.parameters():
          param.requires_grad = False

  if FREEZE_LAYERS>0:
      print(f'Freezing {FREEZE_LAYERS} layers.')
      for layer in model.deberta.encoder.layer[:FREEZE_LAYERS]:
          for param in layer.parameters():
              param.requires_grad = False


# Training Arguments

In [380]:
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    fp16=True,
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    report_to="none",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    overwrite_output_dir=True,
    load_best_model_at_end=True,
    lr_scheduler_type='cosine',
    metric_for_best_model="ents_f5",
    greater_is_better=True,
    weight_decay=0.01
)


ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA devices.

In [ ]:
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Assuming class_weights is a Tensor of weights for each class
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels
        labels = inputs.pop("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        # Reshape for loss calculation
        loss_fct = CrossEntropyLoss(weight=self.class_weights)
        if self.label_smoother is not None and "labels" in inputs:
            loss = self.label_smoother(outputs, inputs)
        else:
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

## training

In [ ]:
if __name__ == "__main__":

  if CFG.DEBUG:
    data=data[0:150]
    args.num_train_epochs = 1
    CFG.max_len=128
    CFG.stride=2
    CFG.trn_folds=[0,1,2]

  tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)
  # tokenizer.add_tokens(['\n', '\r'], special_tokens=True)

  ds = Dataset.from_dict({
      "full_text": [x["full_text"] for x in data],
      "document": [x["document"] for x in data],
      "tokens": [x["tokens"] for x in data],
      "trailing_whitespace": [x["trailing_whitespace"] for x in data],
      "provided_labels": [x["labels"] for x in data],
      "fold": [x["fold"] for x in data]
  })




  data_list=[]
  for example in ds:
      # 各サンプルに対してtokenize関数を呼び出し
      dataset_list=tokenize(example, tokenizer=tokenizer, label2id=label2id)
      for d in dataset_list:
        data_list.append(d)


  ds=Dataset.from_dict({
    'input_ids': [x[0] for x in data_list],
    'token_type_ids': [x[1] for x in data_list],
    "attention_mask":[x[2] for x in data_list],
    'offset_mapping': [x[3] for x in data_list],
    'overflow_to_sample_mapping':[x[4] for x in data_list],
    "labels": [x[5] for x in data_list],
    "length": [x[6] for x in data_list],
    "target_num": [x[7] for x in data_list],
    "group": [x[8] for x in data_list],
    'document':[x[9] for x in data_list],
    'fold':[x[10] for x in data_list],
    'token_map':[x[11] for x in data_list],
    'tokens':[x[12] for x in data_list]
    })

  # ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id}, num_proc=2)

  ds = ds.class_encode_column("group")
  collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
  oof = []
  labels = []
  ds_list=[]
  for fold in CFG.trn_folds:
    ds_i = DatasetDict(
        {
            'train': ds.filter(lambda x: x['fold']!=fold),
            'eval': ds.filter(lambda x: x['fold']==fold),
        }
      )
    ds_list.append(ds_i['eval'])
    ds_i = ds_i.remove_columns(['fold'])
    reference_df_i=reference_df[reference_df['fold']==fold]
    args.output_dir = OUTPUT_DIR + f'fold_{fold}/'

    # train
    model = AutoModelForTokenClassification.from_pretrained(
        CFG.model_name,
        num_labels=len(all_labels),
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )

    freeze_layer(model)

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_i['train'],
        eval_dataset=ds_i['eval'],
        data_collator=collator,
        tokenizer=tokenizer,
        compute_metrics=partial(compute_metrics, id2label=id2label, valid_ds=ds_i['eval'], valid_df=reference_df_i, fold=fold,threshold=0.97)
    )

    trainer.train()

    # eval
    # Make predictions on the validation dataset
    oof_i = trainer.predict(ds_i['eval'])
    oof.append(oof_i.predictions)

    del model,trainer
    torch.cuda.empty_cache()
    gc.collect()

  oof = np.vstack(oof)
  ds_combined=concatenate_datasets(ds_list)
  # Compute the final metrics and log them to Weights & Biases
  print('Computing final metrics...')
  best_f5 = 0.0
  for threshold in [0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.97,0.99,0.995,0.999]:
    print(f'threshold:{threshold}')
    score=compute_metrics((oof,None), id2label, ds_combined, reference_df_i, fold,threshold=threshold)
    f5=score['ents_f5']
    print(f'f5:{f5}, threshold:{threshold}')
    if best_f5 < f5:
      best_f5 = f5
      best_threshold = threshold
  print(f'best_f5:{best_f5}, best_threshold:{best_threshold}')




In [ ]:
runtime.unassign()

In [ ]:
!pip install datasets
from datasets import DatasetDict


In [ ]:

# # DatasetDictを作成
# ds_1 = DatasetDict({'key_1': [1, 2, 3], 'key_2': ['a', 'b', 'c']})
# ds_2 = DatasetDict({'key_1': [4, 5, 6], 'key_2': ['d', 'e', 'f']})
# ds_3 = DatasetDict({'key_1': [7, 8, 9], 'key_2': ['g', 'h', 'i']})

# # リストの中にDatasetDictを含める
# list_of_datasets = [ds_1, ds_2, ds_3]

# # 結合するDatasetDictを初期化
# ds_combined = DatasetDict()

# # リスト内の各DatasetDictを結合
# for ds in list_of_datasets:
#     for key in ds.keys():
#         ds_combined[key] = ds_combined.get(key, []) + ds[key]

# # 結合したDatasetDictを表示
# print(ds_combined)


In [ ]:
# import numpy as np

# # 例として、3つのNumPy配列を作成します
# scores = np.array([5e-1, 1e-7, 4e-4, 1e-6,3e-5])
# log_scores = np.log(scores)
# log_scores/log_scores.sum()



In [ ]:
scores